![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Train Domain-specific Multiclass and Multilabel classifiers

In [0]:
from johnsnowlabs import * 

In [0]:
print(dir(finance))

['AnnotationMerger', 'AssertionDLApproach', 'AssertionDLModel', 'AssertionFilterer', 'AssertionLogRegApproach', 'AssertionLogRegModel', 'BertForSequenceClassification', 'BertForTokenClassification', 'BertSentenceChunkEmbeddings', 'ChunkConverter', 'ChunkFilterer', 'ChunkKeyPhraseExtraction', 'ChunkMapperApproach', 'ChunkMapperFilterer', 'ChunkMapperModel', 'ChunkMergeApproach', 'ChunkMergeModel', 'ChunkSentenceSplitter', 'ClassifierDLApproach', 'ClassifierDLModel', 'Compatibility', 'ContextualParserApproach', 'DateNormalizer', 'DeIdentification', 'DeIdentificationModel', 'DistilBertForSequenceClassification', 'DocumentHashCoder', 'DocumentLogRegClassifierApproach', 'DocumentLogRegClassifierModel', 'EntityChunkEmbeddings', 'FeaturesAssembler', 'GenericClassifierApproach', 'GenericClassifierModel', 'InternalResourceDownloader', 'NerApproach', 'NerChunker', 'NerConverterInternal', 'NerCrfEvaluation', 'NerDLEvaluation', 'NerDLMetrics', 'NerDisambiguator', 'NerDisambiguatorModel', 'NerModel', 'NerQuestionGenerator', 'NorvigSpellEvaluation', 'POSEvaluation', 'RENerChunksFilter', 'ReIdentification', 'RelationExtractionApproach', 'RelationExtractionDLModel', 'RelationExtractionModel', 'ResolverMerger', 'Router', 'SentenceEntityResolverApproach', 'SentenceEntityResolverModel', 'StructuredDeidentification', 'SymSpellEvaluation', 'TFGraphBuilder', 'ZeroShotNerModel', 'ZeroShotRelationExtractionModel', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'log_broken_lib', 'try_import_lib']

# Multilabel classifier training

## Loading the data

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Finance/data/finance_data.csv

dbutils.fs.cp("file:/databricks/driver/finance_data.csv", "dbfs:/") 

Out[3]: True

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/finance_data.csv')
df['label'] = df['label'].apply(eval)

In [0]:
data = spark.createDataFrame(df)

# If you have a single dataset, then split it or else you can load the test dataset the same way that you load the train data.
train, test = data.randomSplit([0.8, 0.2], seed = 123)

In [0]:
train.show(truncate=50)

+--------------------------------------------------+-----------------------------+
 provision| label|
+--------------------------------------------------+-----------------------------+
(a) Consultant or Company may terminate this Pr...| [terminations]|
(a) Effective as of the Effective Date, the Hol...| [waivers, terminations]|
(a) No failure or delay by the Administrative A...| [waivers, amendments]|
(a) No failure or delay by the Agent or any Len...| [waivers, amendments]|
(a) No failure or delay of the Administrative A...| [waivers, amendments]|
(a) The Credit Agreement is, effective as of th...| [amendments]|
(a) The provisions of this Agreement shall be b...| [assigns, successors]|
(a) To induce the other parties hereto to enter...|[representations, warranties]|
(a)  The provisions of this Agreement shall be ...| [assigns, successors]|
(a) All of the representations and warranties m...|[representations, warranties]|
(a) No failure or delay by the Administrative A...| [waivers, amendments]|
(a) No failure or delay by the Administrative A...| [waivers, amendments]|
(a) No failure or delay by the Collateral Agent...| [waivers, amendments]|
(a) No failure or delay of the Agents or any Le...| [waivers, amendments]|
(a) THIS AGREEMENT AND EACH OTHER LOAN DOCUMENT...| [governing laws]|
(a) THIS AGREEMENT WAS NEGOTIATED IN THE STATE ...| [governing laws]|
(a) The provisions of this Agreement shall be b...| [assigns, successors]|
(a) This Agreement may be amended, supplemented...| [waivers, amendments]|
(a) To induce the other parties hereto to enter...|[representations, warranties]|
(a)  The Performance Guarantor hereby waives pr...| [waivers]|
+--------------------------------------------------+-----------------------------+
only showing top 20 rows

In [0]:
from pyspark.sql.functions import col

test.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
 label|count|
+--------------------+-----+
 [governing laws]| 727|
 [counterparts]| 569|
 [notices]| 556|
 [entire agreements]| 556|
 [severability]| 496|
 [survival]| 358|
[assigns, success...| 319|
 [amendments]| 319|
 [expenses]| 248|
 [terminations]| 224|
[waivers, amendme...| 198|
[representations,...| 191|
 [waivers]| 186|
 [assignments]| 186|
 [representations]| 97|
[amendments, enti...| 73|
 [successors]| 64|
 [warranties]| 40|
[amendments, term...| 27|
[assignments, suc...| 10|
+--------------------+-----+
only showing top 20 rows

## With Universal Encoder

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("provision") \
    .setOutputCol("document") \
    .setCleanupMode("shrink")

embeddings = nlp.UniversalSentenceEncoder.pretrained() \
      .setInputCols("document") \
      .setOutputCol("sentence_embeddings")

classsifierdl = nlp.MultiClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("label")\
      .setMaxEpochs(30)\
      .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        embeddings,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 572 ms, sys: 209 ms, total: 781 ms
Wall time: 3min 7s

In [0]:
import os
log_file_name = os.listdir("/dbfs/root/annotator_logs/")[0]

with open("/dbfs/root/annotator_logs/"+log_file_name, "r") as log_file :
    print(log_file.read())

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('label','provision',"class.result").toPandas()
preds_df.head()

Out[13]:

,label,provision,result
0,"[waivers, amendments]",(a) No failure or delay by any Agent or any Le...,"[waivers, amendments]"
1,[assignments],"(a) Seller, the Agent, each Managing Agent, ea...",[assignments]
2,[waivers],(a) Any provision of this Agreement may be wai...,"[waivers, amendments]"
3,[notices],(a) Except where telephonic instructions or no...,[notices]
4,"[governing laws, entire agreements]","(a) THIS AGREEMENT AND ANY CLAIM, CONTROVERSY,...","[governing laws, entire agreements]"


In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

mlb = MultiLabelBinarizer()

y_true = mlb.fit_transform(preds_df['label'])
y_pred = mlb.fit_transform(preds_df['result'])


print("Classification report: \n", (classification_report(y_true, y_pred)))
print("F1 micro averaging:",(f1_score(y_true, y_pred, average='micro')))
print("ROC: ",(roc_auc_score(y_true, y_pred, average="micro")))


Classification report: 
 precision recall f1-score support

 0 0.89 0.77 0.82 643
 1 0.78 0.55 0.64 210
 2 0.82 0.80 0.81 333
 3 0.99 0.98 0.98 579
 4 0.97 0.94 0.95 667
 5 0.95 0.95 0.95 249
 6 0.98 0.98 0.98 747
 7 0.96 0.96 0.96 557
 8 0.89 0.76 0.82 288
 9 0.97 0.94 0.96 515
 10 0.88 0.89 0.88 407
 11 0.94 0.89 0.92 363
 12 0.89 0.74 0.81 256
 13 0.84 0.82 0.83 401
 14 0.83 0.69 0.75 231

 micro avg 0.92 0.88 0.90 6446
 macro avg 0.91 0.84 0.87 6446
weighted avg 0.92 0.88 0.90 6446
 samples avg 0.88 0.89 0.88 6446

F1 micro averaging: 0.8992507572134545
ROC: 0.9345512577363616

## With Bert Embeddings

We do not have have any specific Financial Sentence Embeddings, but we can use Financial Bert Embeddings and then average them.

In [0]:
embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
          .setInputCols(["document", "token"]) \
          .setOutputCol("embeddings")

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("provision") \
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddingsSentence = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierdl = nlp.MultiClassifierDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("label")\
    .setMaxEpochs(1)\
    .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        embeddings,
        embeddingsSentence,
        classsifierdl
    ])

In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 9.01 s, sys: 4.68 s, total: 13.7 s
Wall time: 49min 1s

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('provision','label',"class.result").toPandas()

In [0]:
preds_df.head()

Out[12]:

,provision,label,result
0,(a) No failure or delay by any Agent or any Le...,"[waivers, amendments]",[waivers]
1,"(a) Seller, the Agent, each Managing Agent, ea...",[assignments],[assignments]
2,(a) Any provision of this Agreement may be wai...,[waivers],[waivers]
3,(a) Except where telephonic instructions or no...,[notices],[notices]
4,"(a) THIS AGREEMENT AND ANY CLAIM, CONTROVERSY,...","[governing laws, entire agreements]",[entire agreements]


In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

mlb = MultiLabelBinarizer()

y_true = mlb.fit_transform(preds_df['label'])
y_pred = mlb.fit_transform(preds_df['result'])


print("Classification report: \n", (classification_report(y_true, y_pred)))
print("F1 micro averaging:",(f1_score(y_true, y_pred, average='micro')))
print("ROC: ",(roc_auc_score(y_true, y_pred, average="micro")))


Classification report: 
 precision recall f1-score support

 0 0.98 0.67 0.80 643
 1 0.88 0.38 0.53 210
 2 0.93 0.39 0.55 333
 3 0.99 0.98 0.99 579
 4 0.99 0.96 0.97 667
 5 1.00 0.94 0.97 249
 6 0.98 0.97 0.98 747
 7 0.99 0.94 0.97 557
 8 0.90 0.90 0.90 288
 9 0.99 0.97 0.98 515
 10 0.88 0.90 0.89 407
 11 0.97 0.85 0.90 363
 12 0.90 0.81 0.85 256
 13 0.89 0.79 0.84 401
 14 0.83 0.70 0.76 231

 micro avg 0.96 0.85 0.90 6446
 macro avg 0.94 0.81 0.86 6446
weighted avg 0.95 0.85 0.89 6446
 samples avg 0.90 0.87 0.88 6446

F1 micro averaging: 0.8969946480032935
ROC: 0.9208592268995528

### Saving & loading back the trained model

In [0]:
clf_pipelineModel.stages

Out[21]: [DocumentAssembler_0befb157d5b5,
 REGEX_TOKENIZER_845e9cea52a1,
 BERT_EMBEDDINGS_29ce72cd673e,
 SentenceEmbeddings_c13729f7bf05,
 MultiClassifierDLModel_fb74a81172fa]

In [0]:
clf_pipelineModel.stages[-1].write().overwrite().save('/dbfs/MultilabelClfBert')

In [0]:
# Load back  saved Multilabel Classifier Model
MultilabelClfModel = nlp.MultiClassifierDLModel.load('/dbfs/MultilabelClfBert')

In [0]:
ld_pipeline = Pipeline(stages=[document_assembler, tokenizer, embeddings, embeddingsSentence, MultilabelClfModel])
ld_pipeline_model = ld_pipeline.fit(spark.createDataFrame([['']]).toDF("provision"))

In [0]:
# Apply Model Transform to testData
ld_preds = ld_pipeline_model.transform(test)

In [0]:
ld_preds_df = ld_preds.select('provision','label',"class.result").toPandas()

In [0]:
ld_preds_df.head(10)

,provision,label,result
0,"(a) Effective as of the Effective Date, the Ho...","[waivers, terminations]",[waivers]
1,(a) No failure or delay by the Administrative ...,"[waivers, amendments]","[waivers, amendments]"
2,(a) No failure or delay on the part of any par...,"[waivers, amendments]",[waivers]
3,"(a) Seller, the Agent, each Managing Agent, ea...",[assignments],[assignments]
4,(a) The provisions of this Agreement shall be ...,"[assigns, successors]","[successors, assigns]"
5,(a) No failure or delay of the Administrative...,"[waivers, amendments]","[waivers, amendments]"
6,(a) All of the representations and warranties ...,"[representations, warranties]","[warranties, representations]"
7,(a) Any Lender may at any time assign to one o...,[assignments],[assignments]
8,(a) Each of the Borrower and the Parent hereby...,"[representations, warranties]","[warranties, representations]"
9,(a) Except as otherwise expressly provided her...,[notices],[notices]


# Multiclass classifier training

## Loading the data

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Finance/data/finance_clf_data.csv

dbutils.fs.cp("file:/databricks/driver/finance_clf_data.csv", "dbfs:/") 

Out[3]: True

In [0]:
import pandas as pd
df = pd.read_csv('/dbfs/finance_clf_data.csv')

In [0]:
df.head()

Out[5]:

,text,label,len
0,Presently we do not believe any U S or State r...,business,402
1,\r\nnetwork outages or performance degradatio...,risk_factors,496
2,Available Information\r\nOur reports filed wit...,business,356
3,\r\n 42 530\r\n \r\n \r\n \r\n \r\n \r\n 42 5...,financial_statements,359
4,8\r\nTable of Contents\r\ndevelopment employee...,business,582


In [0]:
df['label'].value_counts()

Out[6]: risk_factors 1926
financial_statements 1888
business 970
financial_conditions 346
form_10k_summary 240
executives_compensation 155
controls_procedures 138
equity 111
market_risk 100
executives 73
legal_proceedings 51
properties 48
security_ownership 46
exhibits 36
Name: label, dtype: int64

In [0]:
data = spark.createDataFrame(df)

train, test = data.randomSplit([0.8, 0.2], seed = 100)

In [0]:
from pyspark.sql.functions import col

train.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
 label|count|
+--------------------+-----+
 risk_factors| 1529|
financial_statements| 1491|
 business| 769|
financial_conditions| 267|
 form_10k_summary| 196|
executives_compen...| 128|
 controls_procedures| 107|
 equity| 92|
 market_risk| 75|
 executives| 58|
 properties| 39|
 legal_proceedings| 36|
 security_ownership| 34|
 exhibits| 31|
+--------------------+-----+

In [0]:
from pyspark.sql.functions import col

test.groupBy("label") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-----+
 label|count|
+--------------------+-----+
 risk_factors| 397|
financial_statements| 397|
 business| 201|
financial_conditions| 79|
 form_10k_summary| 44|
 controls_procedures| 31|
executives_compen...| 27|
 market_risk| 25|
 equity| 19|
 executives| 15|
 legal_proceedings| 15|
 security_ownership| 12|
 properties| 9|
 exhibits| 5|
+--------------------+-----+

## With Universal Encoder

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document") 

embeddings = nlp.UniversalSentenceEncoder.pretrained() \
    .setInputCols("document") \
    .setOutputCol("sentence_embeddings")

classsifierdl = finance.ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("label")\
    .setMaxEpochs(60)\
    .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        embeddings,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 94.3 ms, sys: 43.8 ms, total: 138 ms
Wall time: 27.6 s

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('label','text',"class.result").toPandas()
preds_df.head()

Out[40]:

,label,text,result
0,risk_factors,\r\n\r\n \r\n\r\n\r\nNet cash provided by oper...,[financial_statements]
1,financial_statements,\r\n\r\n\r\n \r\n \r\n \r\n Identification of...,[financial_statements]
2,form_10k_summary,\r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n\...,[financial_statements]
3,financial_statements,\r\n \r\n 120\r\n \r\n \r\n \r\n 202\r\n \r\n...,[financial_statements]
4,risk_factors,\r\n \r\nAn assertion by a third party that w...,[risk_factors]


In [0]:
# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))

precision recall f1-score support

 business 0.70 0.76 0.73 201
 controls_procedures 0.00 0.00 0.00 31
 equity 0.00 0.00 0.00 19
 executives 0.00 0.00 0.00 15
executives_compensation 0.00 0.00 0.00 27
 exhibits 0.00 0.00 0.00 5
 financial_conditions 0.00 0.00 0.00 79
 financial_statements 0.64 0.93 0.76 397
 form_10k_summary 0.00 0.00 0.00 44
 legal_proceedings 0.00 0.00 0.00 15
 market_risk 0.00 0.00 0.00 25
 properties 0.00 0.00 0.00 9
 risk_factors 0.73 0.87 0.79 397
 security_ownership 0.00 0.00 0.00 12

 accuracy 0.68 1276
 macro avg 0.15 0.18 0.16 1276
 weighted avg 0.53 0.68 0.60 1276

### Saving & loading back the trained model

In [0]:
clf_pipelineModel.stages

Out[45]: [DocumentAssembler_23451413b4e2,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 FinanceClassifierDLModel_7428e5236582]

In [0]:
clf_pipelineModel.stages[-1].write().overwrite().save('/dbfs/Clf_Use')

In [0]:
# Load back  saved Classifier Model
ClfModel = finance.ClassifierDLModel.load('/dbfs/Clf_Use')

In [0]:
ld_pipeline = Pipeline(stages=[document_assembler, embeddings,ClfModel])
ld_pipeline_model = ld_pipeline.fit(spark.createDataFrame([['']]).toDF("text"))

In [0]:
# Apply Model Transform to testData
ld_preds = ld_pipeline_model.transform(test)

In [0]:
ld_preds_df = ld_preds.select('text','label',"class.result").toPandas()

In [0]:
ld_preds_df.head()

Out[51]:

,text,label,result
0,\r\n\r\n \r\n\r\n\r\nNet cash provided by oper...,risk_factors,[financial_statements]
1,\r\n\r\n\r\n \r\n \r\n \r\n Identification of...,financial_statements,[financial_statements]
2,\r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n\...,form_10k_summary,[financial_statements]
3,\r\n \r\n 120\r\n \r\n \r\n \r\n 202\r\n \r\n...,financial_statements,[financial_statements]
4,\r\n \r\nAn assertion by a third party that w...,risk_factors,[risk_factors]


## With Bert Embeddings

We do not have Financial Sentence Embeddings yet, But we can use the Financial Word Embeddings and then average them.

In [0]:
embeddings = nlp.BertEmbeddings.pretrained("bert_embeddings_sec_bert_base","en") \
    .setInputCols(["document", "token"]) \
    .setOutputCol("embeddings")

bert_embeddings_sec_bert_base download started this may take some time.
Approximate size to download 390.4 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]

In [0]:
document_assembler = nlp.DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

embeddingsSentence = nlp.SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierdl = finance.ClassifierDLApproach() \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("class") \
    .setLabelColumn("label")\
    .setMaxEpochs(8)\
    .setLr(0.001)\
    .setEnableOutputLogs(True)

clf_pipeline = Pipeline(
    stages = [
        document_assembler,
        tokenizer,
        embeddings,
        embeddingsSentence,
        classsifierdl
    ])

In [0]:
%%time
clf_pipelineModel = clf_pipeline.fit(train)

CPU times: user 1.31 s, sys: 615 ms, total: 1.93 s
Wall time: 6min 7s

In [0]:
preds = clf_pipelineModel.transform(test)

In [0]:
preds_df = preds.select('label','text',"class.result").toPandas()

In [0]:
preds_df.head()

Out[57]:

,label,text,result
0,risk_factors,\r\n\r\n \r\n\r\n\r\nNet cash provided by oper...,[financial_statements]
1,financial_statements,\r\n\r\n\r\n \r\n \r\n \r\n Identification of...,[financial_statements]
2,form_10k_summary,\r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n \r\n\...,[financial_statements]
3,financial_statements,\r\n \r\n 120\r\n \r\n \r\n \r\n 202\r\n \r\n...,[financial_statements]
4,risk_factors,\r\n \r\nAn assertion by a third party that w...,[risk_factors]


In [0]:
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['label'], preds_df['result']))


precision recall f1-score support

 business 0.00 0.00 0.00 201
 controls_procedures 0.00 0.00 0.00 31
 equity 0.00 0.00 0.00 19
 executives 0.00 0.00 0.00 15
executives_compensation 0.00 0.00 0.00 27
 exhibits 0.00 0.00 0.00 5
 financial_conditions 0.00 0.00 0.00 79
 financial_statements 0.56 0.98 0.71 397
 form_10k_summary 0.00 0.00 0.00 44
 legal_proceedings 0.00 0.00 0.00 15
 market_risk 0.00 0.00 0.00 25
 properties 0.00 0.00 0.00 9
 risk_factors 0.65 0.95 0.77 397
 security_ownership 0.00 0.00 0.00 12

 accuracy 0.60 1276
 macro avg 0.09 0.14 0.11 1276
 weighted avg 0.38 0.60 0.46 1276

# Save model

In [0]:
# Save a Spark NLP model
clf_pipelineModel.stages[-1].write().overwrite().save('/dbfs/ClfBert')